In [10]:
!pip install pycaw

In [1]:
import cv2
import mediapipe as mp
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import math

# Initialize VideoCapture
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=2,
                       min_detection_confidence=0.7,
                       min_tracking_confidence=0.5)

# Initialize MediaPipe Drawing
mp_drawing = mp.solutions.drawing_utils

# Initialize the system's volume control
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, 0x17, None)
volume = interface.QueryInterface(IAudioEndpointVolume)

# Define the volume control thresholds
VOLUME_MIN = 0.0
VOLUME_MAX = 1.0
VOLUME_INCREMENT = 0.02

# Define the volume bar position and size
volume_bar_x = 50
volume_bar_y = 50
volume_bar_width = 400
volume_bar_height = 20

# Main loop
while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    # Perform hand detection and landmarks recognition
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Check for hand landmarks and perform volume control
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get the thumb and index finger landmarks
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

            # Calculate the distance between thumb and index finger
            thumb_x = int(thumb_tip.x * image.shape[1])
            thumb_y = int(thumb_tip.y * image.shape[0])
            index_x = int(index_finger_tip.x * image.shape[1])
            index_y = int(index_finger_tip.y * image.shape[0])
            distance = math.sqrt((thumb_x - index_x) ** 2 + (thumb_y - index_y) ** 2)

            # Draw a line connecting thumb and index finger
            cv2.line(image, (thumb_x, thumb_y), (index_x, index_y), (255, 0, 0), 3)

            # Calculate the volume change based on the distance between fingers
            volume_change = (distance / 100) * VOLUME_INCREMENT

            # Get the current volume
            current_volume = volume.GetMasterVolumeLevelScalar()

            # Calculate the new volume level
            new_volume = max(VOLUME_MIN, min(current_volume + volume_change, VOLUME_MAX))

            # Set the new volume
            volume.SetMasterVolumeLevelScalar(new_volume, None)

            # Draw the volume bar
            volume_bar_width_filled = int(volume_bar_width * new_volume)
            cv2.rectangle(image, (volume_bar_x, volume_bar_y),
                          (volume_bar_x + volume_bar_width, volume_bar_y + volume_bar_height),
                          (0, 0, 0), -1)
            cv2.rectangle(image, (volume_bar_x, volume_bar_y),
                          (volume_bar_x + volume_bar_width_filled, volume_bar_y + volume_bar_height),
                          (0, 255, 0), -1)

    # Display the image
    cv2.imshow('Volume Control', image)

    # Check for 'q' key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture and close all windows
cap.release()
cv2.destroyAllWindows()